In [ ]:
import pandas as pd

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics
import numpy as np

In [ ]:
from multiprocessing import Pool

In [ ]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

In [ ]:
flash_names = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*dash\\input\\*', 'cs')

In [ ]:
info_dict = {}

In [ ]:
gn = global_names = preproc_tool.GlobalNames()

In [ ]:
this_file = pd.read_csv(flash_names[0], index_col = [0], parse_dates = True)
this_file = preproc_tool.rename_columns_by_dict(this_file)
this_file.head()
info_dict['Начало записи данных'] = [this_file.index[0]]
info_dict['Конец записи данных'] = [this_file.index[-1]]
info_dict['Продолжительность записи данных'] = [this_file.index[-1] - this_file.index[0]]
info_dict['Общее число записей'] =   [this_file.shape[0]]
info_dict['Средняя дискретность'] =   [(this_file.index[-1] - this_file.index[0])/this_file.shape[0]]
this_file_work = this_file[this_file[gn.freq_hz]>0]
info_dict['Медианное значение загрузки во время работы'] =   [ this_file[gn.motor_load_perc].median()]
info_dict['Медианное значение тока во время работы'] =   [this_file[gn.i_a_motor_a].median()]
info_dict['Медианное значение мощности во время работы'] =   [this_file[gn.active_power_kwt].median()]
info_dict['Медианное значение частоты во время работы'] =   [this_file[gn.freq_hz].median()]
info_dict['Максимальное значение загрузки во время работы'] =   [ this_file[gn.motor_load_perc].max()]
info_dict['Максимальное значение тока во время работы'] =   [this_file[gn.i_a_motor_a].max()]
info_dict['Максимальное значение мощности во время работы'] =   [this_file[gn.active_power_kwt].max()]
info_dict['Максимальное значение частоты во время работы'] =   [this_file[gn.freq_hz].max()]

In [ ]:
info_dict

In [ ]:
this_file.columns

In [ ]:
this_file_work.describe()

In [ ]:
this_file_work.median()